In [ ]:
import argparse
import os
import torch
import numpy as np
import cv2
import sys
import shutil
import matplotlib.pyplot as plt

from attrdict import AttrDict
import pandas as pd

from torch.autograd import Variable
from sgan.data.loader import data_loader
from sgan.models import CNNLSTM1, CNNLSTM2, CNNLSTM3

model_path = "./models/cnnlstm2-vgg16-lausanne-noembedder_with_model.pt" 
dataset = "../dataset/val/lausanne"

def create_folder(directory, renew=False):
    
    # if directory doesn't exist
    if not os.path.exists(directory):
        os.makedirs(directory)
    
    # if already exist, delete and re-create
    elif os.path.exists(directory) and renew:
        shutil.rmtree(directory)
        os.makedirs(directory)

def get_classifier(checkpoint):
        
    args = AttrDict(checkpoint['args'])  
    classifier = CNNLSTM2(
        embedding_dim=args.embedding_dim,
        h_dim=args.h_dim,
        dropout=0.0,
        grad=True)
    classifier.load_state_dict(checkpoint['best_state'])
    return classifier

def save_gradients(gradients, pedestrian_foldernames, pedestrian_filenames, result):
                
    create_folder(os.path.join(os.getcwd(), "results", result, os.path.basename(pedestrian_foldernames[0][0])))    
    for g,foldername,filename in zip(gradients, pedestrian_foldernames[0], pedestrian_filenames[0]):
        print(os.path.join(os.getcwd(), "results", result, os.path.basename(foldername), filename))
        cv2.imwrite(os.path.join(os.getcwd(), "results", result, os.path.basename(foldername), filename), np.uint8(g*255))

def save_classification(pedestrian_images, decision, pedestrian_filenames, result):
    
    pedestrian_images = np.transpose(pedestrian_images, axes=(0,2,3,1))
        
    # get the filename
    pedestrian_filenames = [[f.split("/")[-4], f.split("/")[-2], f.split("/")[-1]] for f in pedestrian_filenames] 
    # create folder for pedestrian
    create_folder(os.path.join(os.getcwd(), "results", pedestrian_filenames[0][0], result, pedestrian_filenames[0][1]))
    
    for i,f,d in zip(pedestrian_images, pedestrian_filenames,decision):
        print(f, d, result)
        cv2.imwrite(os.path.join(os.getcwd(), "results", f[0], result, f[1], f[2][0:-4]+"-"+str(d)+".png"), np.uint8(i*255))
        
def guidedbackprop(args, loader, classifier):
       
    tp = 0
    tn = 0
    fp = 0
    fn = 0
        
    classifier.train()        
    for batch in loader:

        (pedestrian_crops, decision_true, pedestrian_foldernames, pedestrian_filenames) = batch
        
        print(decision_true.item())
        
        # (str(pedestrian_foldernames[0][0]) == "crops/Ouchy-2-Left/0000000133")
        # (str(pedestrian_foldernames[0][0]) == "crops/Ouchy-2-Left/0000000159")
        if(0):
            
            decision_pred_scores = classifier(pedestrian_crops, input_as_var=True, classify_every_timestep=False)        
            if(torch.cuda.is_available()):
                decision_onehot = torch.cuda.FloatTensor(1, decision_pred_scores.size()[-1]).zero_()
            else:
                decision_onehot = torch.FloatTensor(1, decision_pred_scores.size()[-1]).zero_()
            decision_onehot[0][decision_pred_scores.max(1)[1]] = 1
            if(torch.cuda.is_available()):
                decision_onehot = decision_onehot.cuda()
            else:
                decision_onehot = decision_onehot
                
            # backprop
            classifier.zero_grad()
            decision_pred_scores.backward(gradient=decision_onehot)
            gradients = classifier.gradients.cpu().numpy()

            # normalize each image
            gradients_min = np.amin(np.reshape(gradients, newshape=(np.shape(gradients)[0],-1)), 1)
            gradients_max = np.amax(np.reshape(gradients, newshape=(np.shape(gradients)[0],-1)), 1)
            gradients -= gradients_min[:,None,None,None]
            gradients /= gradients_max[:,None,None,None] - gradients_min[:,None,None,None]
            gradients = np.transpose(gradients, (0, 2, 3, 1))

            # save images to folder
            decision_true = decision_true.item()
            decision_pred = decision_pred_scores.max(1)[1].item()

            print(str(pedestrian_foldernames[0][0]), decision_pred, decision_true)
            
            # tn
            if(decision_pred == 0 and decision_true == 0):
                tn+=1
                #save_gradients(gradients, pedestrian_foldernames, pedestrian_filenames, "tn")            
            # fn
            if(decision_pred == 0 and decision_true == 1):
                fn+=1
                #save_gradients(gradients, pedestrian_foldernames, pedestrian_filenames, "fn")            
            # fp
            if(decision_pred == 1 and decision_true == 0):
                fp+=1
                #save_gradients(gradients, pedestrian_foldernames, pedestrian_filenames, "fp")            
            # tp
            if(decision_pred == 1 and decision_true == 1):
                tp+=1
                #save_gradients(gradients, pedestrian_foldernames, pedestrian_filenames, "tp")

    print(tn,fn,fp,tp)
                
# only works for batch size of 1
def classify_every_timestep(args, loader, classifier):
       
    # initialize results dict
    images_dict = dict()
    prediction_dict = dict()
        
    classifier.eval()        
    for batch in loader:

        (pedestrian_crops, decision_true, pedestrian_foldernames, pedestrian_filenames) = batch
        
        # (str(pedestrian_foldernames[0][0]) == "crops/Ouchy-2-Left/0000000159") 
        # (str(pedestrian_foldernames[0][0]) == "crops/Ouchy-2-Left/0000000133")
        if(1):
                         
            # predict decision
            decision = classifier(pedestrian_crops, input_as_var=False, classify_every_timestep=True)        
            decision_true = decision_true.item()
            decision_pred = torch.stack(decision).cpu().numpy()[-1]
            
            # update dict
            images_dict[str(pedestrian_foldernames[0][0])] = [str(pedestrian_filenames[0][0])]
            prediction_dict[str(pedestrian_foldernames[0][0])] = [decision_pred[0]]
            for i in range(1,len(decision_pred)):
                images_dict[str(pedestrian_foldernames[0][0])].append(str(pedestrian_filenames[0][i]))
                prediction_dict[str(pedestrian_foldernames[0][0])].append(decision_pred[i])
        
    return images_dict, prediction_dict
            
def main(args):
            
    # create results folder if it does not exist
    # - used for guidedbackprop
    results_root = os.path.join(os.getcwd(), "results")
    create_folder(results_root, renew=False)
    
    # model path
    path = args.model_path
    print("model path ", path)

     # get the model
    checkpoint = torch.load(path, map_location='cpu')
    classifier = get_classifier(checkpoint)

    # generate the arguments
    _args = AttrDict(checkpoint['args']) 
    _args.batch_size = 1
    _args.timestep = 10
    print(_args)
    input()

    # get the dataset
    val_path = os.path.join(args.dataset, "val") 
    dset, loader = data_loader(_args, val_path, "val")
    
    print('Running Guided Backprop')
    guidedbackprop(_args, loader, classifier)

    #print('Classifying at every timestep')
    #images_dict, prediction_dict = classify_every_timestep(_args, loader, classifier)
    #return images_dict, prediction_dict
        
if __name__ == '__main__':
    parser = argparse.ArgumentParser()
    parser.add_argument('--model_path', type=str)
    parser.add_argument('--dset_type', default='test', type=str)

    parser.model_path = model_path
    parser.dataset = dataset #"./datasets/lausanne" #"./datasets/riponne" lausanne
    args = parser
    images_dict, prediction_dict = main(args)

In [ ]:
df = pd.read_csv("./datasets/riponne/val/annotations/Riponne-Left-final.txt") 
for count,((folderpath1, filename), (folderpath2, prediction)) in enumerate(zip(images_dict.items(), prediction_dict.items())):

    # get the filenames (images) in the dataframe for the current pedestrian
    decision = 0
    for f in df.loc[(df["folderpath"] == folderpath1)]["filename"]:
        # if the image has been given a label by the classifier
        if(f in filename):
            i = filename.index(f) # index at which the filename occurs
            p = prediction[i]     # get the prediction given the index
            #print(f, p)
            decision = p
            df.loc[(df["folderpath"] == folderpath1) & (df["filename"] == f), "cross_pred"] = decision
        else:
            # set it to the previous decision if it is not in the list of predicted images
            df.loc[(df["folderpath"] == folderpath1) & (df["filename"] == f), "cross_pred"] = decision
df.to_csv("Riponne.txt", sep=',', encoding='utf-8')    

In [ ]:
import os
import sys
import cv2
import glob
import shutil
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from tqdm import tqdm

# ------- args ------- # 
in_video    = "./datasets/lausanne/val/videos/Ouchy-2-Left-cropped.MP4" 
op_video    = in_video[0:-4] + "-pred.MP4"    # "./videos/23-10-2018/Ouchy-2-Left-cropped-pred.MP4"
annotations = "./Ouchy-last-timestep.txt"

fontface = cv2.FONT_HERSHEY_SIMPLEX;
scale = 0.4;
thickness = 1;
baseline = 0;

x_offset = 0
y_offset = 0
# ------- args ------- #

# ------- main ------- #
df = pd.read_csv(annotations)

in_cap = cv2.VideoCapture(in_video)
op_cap = cv2.VideoWriter(op_video, cv2.VideoWriter_fourcc(*'XVID'), 30, (int(in_cap.get(3)),int(in_cap.get(4))))
       
print("Drawing trajectories")
pbar = tqdm(total = df["frame"].max())
t = 1
while(in_cap.isOpened()):
            
    pbar.update(1)

    #if(t==df["frame"].max()):
    #    break
    
    # get frame
    ret, im1 = in_cap.read()
    im = np.copy(im1)
    if(ret==False):
        break

    for row in df[df["frame"] == t].itertuples(index=True, name='Pandas'):        
        #if((row.height <= 50) or (row.tly + row.height + 40 > np.shape(im)[0])):
        # red box for crossing
        if(row.cross_pred != -1):# and row.x > min_lifetime_crossers):
                    
            # put ground truth
            #label = "true: " + str(row.cross_true)
            #text = cv2.getTextSize(label, fontface, scale, thickness);
            #cv2.rectangle(im, (x_offset+row.tlx, y_offset+row.height+row.tly-20), (x_offset+row.tlx+row.width+20, y_offset+row.height+row.tly-10), (0,0,0), -1);
            #cv2.putText(im, label, (x_offset+row.tlx, y_offset+row.height+row.tly-10), fontface, scale, (255,255,255), thickness, 8);
                    
            # put prediction
            #label = "pred: " + str(row.cross_pred)
            #text = cv2.getTextSize(label, fontface, scale, thickness);
            #cv2.rectangle(im, (x_offset+row.tlx, y_offset+row.height+row.tly-10), (x_offset+row.tlx+row.width+20, y_offset+row.height+row.tly), (0,0,0), -1);
            #cv2.putText(im, label, (x_offset+row.tlx, y_offset+row.height+row.tly), fontface, scale, (255,255,255), thickness, 8);            
            
            # put ID
            #label = str(row.id)
            #text = cv2.getTextSize(label, fontface, scale, thickness);
            #cv2.rectangle(im, (x_offset+row.tlx, y_offset+row.tly+row.height), (x_offset+row.tlx+row.width+20, y_offset+row.tly+row.height+10), (0,0,0), -1);
            #cv2.putText(im, label, (x_offset+row.tlx, y_offset+row.tly+row.height+10), fontface, scale, (255,255,255), thickness, 8);  
                    
            # draw bounding box
            if(row.cross_pred == 1):
                cv2.rectangle(im, (x_offset+row.tlx, y_offset+row.tly), (x_offset+row.tlx+row.width, y_offset+row.tly+row.height), color=(0, 0, 255), thickness=2)
            
            if(row.cross_pred == 0):
                cv2.rectangle(im, (x_offset+row.tlx, y_offset+row.tly), (x_offset+row.tlx+row.width, y_offset+row.tly+row.height), color=(0, 255, 0), thickness=2)
                   
    op_cap.write(im)
    t+=1
    #if(t==100):
    #    break
    
in_cap.release()
op_cap.release() 
# ------- main ------- #

In [ ]:
import os
import sys
import cv2
import glob
import shutil
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from tqdm import tqdm

# ------- args ------- # 
in_video    = "./datasets/riponne/val/videos/Riponne-Left-cropped.MP4" 
op_video    = in_video[0:-4] + "-pred.MP4"    # "./videos/23-10-2018/Ouchy-2-Left-cropped-pred.MP4"
annotations = "./Riponne.txt"

fontface = cv2.FONT_HERSHEY_SIMPLEX;
scale = 0.4;
thickness = 1;
baseline = 0;

x_offset = 0
y_offset = 0
# ------- args ------- #

# ------- main ------- #
df = pd.read_csv(annotations)

in_cap = cv2.VideoCapture(in_video)
op_cap = cv2.VideoWriter(op_video, cv2.VideoWriter_fourcc(*'XVID'), 30, (int(in_cap.get(3)),int(in_cap.get(4))))
       
print("Drawing trajectories")
pbar = tqdm(total = df["frame"].max())
t = 1
while(in_cap.isOpened()):
            
    pbar.update(1)

    #if(t==df["frame"].max()):
    #    break
    
    # get frame
    ret, im1 = in_cap.read()
    im = np.copy(im1)
    if(ret==False):
        break

    for row in df[df["frame"] == t].itertuples(index=True, name='Pandas'):        
        #if((row.height <= 50) or (row.tly + row.height + 40 > np.shape(im)[0])):
        # red box for crossing
        if(row.cross_pred != -1):# and row.x > min_lifetime_crossers):
                    
            # put ground truth
            #label = "true: " + str(row.cross_true)
            #text = cv2.getTextSize(label, fontface, scale, thickness);
            #cv2.rectangle(im, (x_offset+row.tlx, y_offset+row.height+row.tly-20), (x_offset+row.tlx+row.width+20, y_offset+row.height+row.tly-10), (0,0,0), -1);
            #cv2.putText(im, label, (x_offset+row.tlx, y_offset+row.height+row.tly-10), fontface, scale, (255,255,255), thickness, 8);
                    
            # put prediction
            #label = "pred: " + str(row.cross_pred)
            #text = cv2.getTextSize(label, fontface, scale, thickness);
            #cv2.rectangle(im, (x_offset+row.tlx, y_offset+row.height+row.tly-10), (x_offset+row.tlx+row.width+20, y_offset+row.height+row.tly), (0,0,0), -1);
            #cv2.putText(im, label, (x_offset+row.tlx, y_offset+row.height+row.tly), fontface, scale, (255,255,255), thickness, 8);            
            
            # put ID
            #label = str(row.id)
            #text = cv2.getTextSize(label, fontface, scale, thickness);
            #cv2.rectangle(im, (x_offset+row.tlx, y_offset+row.tly+row.height), (x_offset+row.tlx+row.width+20, y_offset+row.tly+row.height+10), (0,0,0), -1);
            #cv2.putText(im, label, (x_offset+row.tlx, y_offset+row.tly+row.height+10), fontface, scale, (255,255,255), thickness, 8);  
                    
            # draw bounding box
            if(row.cross_pred == 1):
                cv2.rectangle(im, (x_offset+row.tlx, y_offset+row.tly), (x_offset+row.tlx+row.width, y_offset+row.tly+row.height), color=(0, 0, 255), thickness=2)
            
            if(row.cross_pred == 0):
                cv2.rectangle(im, (x_offset+row.tlx, y_offset+row.tly), (x_offset+row.tlx+row.width, y_offset+row.tly+row.height), color=(0, 255, 0), thickness=2)
                   
    op_cap.write(im)
    t+=1
    #if(t==100):
    #    break
    
in_cap.release()
op_cap.release() 
# ------- main ------- #

In [ ]:
checkpoint = torch.load("models/cnnlstm2-vgg16-lausanne-noembedder_with_model.pt", map_location='cpu')
print(checkpoint['best_state'])

In [ ]:
for key, value in checkpoint.items() :
    print(key)
    
print(checkpoint['best_t'])
for key, value in checkpoint['best_state'].items() :
    if(key == "lstm.weight_hh_l0"):
        a = value
        
for name, param in classifier.named_parameters():
    if(name == "lstm.weight_hh_l0"):
        b = param
for name, param in classifier.named_parameters():
    print(name)
print(np.linalg.norm((a-b).detach().numpy()))

In [ ]:
for p in classifier.named_children():
    #if(p == 'lstm'):
    print(p)

In [ ]:
print(checkpoint['metrics_val']['d_precision'][57])

In [ ]:
import os
import torch
import numpy as np
import cv2
import matplotlib.pyplot as plt

checkpoint = torch.load("./models/ckpt_with_model.pt", map_location='cpu')
plt.figure()
plt.title("loss")
plt.plot(checkpoint['metrics_train']['d_loss'], 'r', checkpoint['metrics_val']['d_loss'], 'b')
plt.legend(["train", "val"])
plt.figure()
plt.title("precision")
plt.plot(checkpoint['metrics_train']['d_precision'], 'r', checkpoint['metrics_val']['d_precision'], 'b')
plt.legend(["train", "val"])
plt.figure()
plt.title("recall")
plt.plot(checkpoint['metrics_train']['d_recall'], 'r', checkpoint['metrics_val']['d_recall'], 'b')
plt.legend(["train", "val"])

print(che)

In [ ]:
checkpoint = torch.load("./vgg16-lausanne.pt", map_location='cpu')
print(checkpoint['best_state'])

In [ ]:
checkpoint = torch.load("./vgg16-lausanne.pt", map_location='cpu')
print(checkpoint['best_state'])
plt.figure()
plt.title("loss")
plt.plot(checkpoint['metrics_train']['d_loss'], 'r', checkpoint['metrics_val']['d_loss'], 'b')
plt.legend(["train", "val"])
plt.figure()
plt.title("precision")
plt.plot(checkpoint['metrics_train']['d_precision'], 'r', checkpoint['metrics_val']['d_precision'], 'b')
plt.legend(["train", "val"])
plt.figure()
plt.title("recall")
plt.plot(checkpoint['metrics_train']['d_recall'], 'r', checkpoint['metrics_val']['d_recall'], 'b')
plt.legend(["train", "val"])

In [ ]:
import pandas as pd

df = pd.read_csv("Ouch.txt")
df = df[(df["folderpath"] == "crops/Ouchy-2-Left/0000000645") & (df["cross_pred"] == 1)]
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(df)